<a href="https://colab.research.google.com/github/zzanggyu/Pill-AI-Notification-System/blob/main/API_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, jsonify
import requests
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

# 모델 서버 API
MODEL_SERVER_URL = "http://localhost:5000/process_image"
# 의약품 낱알식별정보 API
PILL_IDENTIFICATION_API_URL = "http://apis.data.go.kr/1471000/MdcinGrnIdntfcInfoService01/getMdcinGrnIdntfcInfoList01"
# 식약처 e약은요 api
PILL_INFO_API_URL = "http://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList"

# API 키 (실제 사용 시 환경 변수로 관리하는 것이 좋습니다)
API_KEY = "yzl64iURh2NXaF8F3PabeI+BkGb4gf/Facx4EItEsWPxi5A9j+LQj5xdxqZ/w719cr/Mv+QHScdDQQnZoc3s7g=="

@app.route('/analyze_pill', methods=['POST'])
def analyze_pill():
    if 'image' not in request.json:
        return jsonify({'error': 'No image data provided'}), 400

    image_data = request.json['image']

    try:
        # 1. 모델 서버에 요청 전송
        model_response = requests.post(MODEL_SERVER_URL, json={'image': image_data})
        model_response.raise_for_status()
        model_results = model_response.json()['results']

        # 2. 각 알약에 대해 식별 및 정보 조회
        final_results = []
        for pill_result in model_results:
            identified_pill = identify_pill(pill_result)
            pill_info = get_pill_info(identified_pill)
            final_results.append({
                'pill_number': pill_result['pill_number'],
                'model_result': pill_result,
                'identified_pill': identified_pill,
                'pill_info': pill_info
            })

        return jsonify(final_results), 200

    except requests.RequestException as e:
        return jsonify({'error': f'Error communicating with external server: {str(e)}'}), 500
    except Exception as e:
        return jsonify({'error': f'Unexpected error: {str(e)}'}), 500

def identify_pill(pill_result):
    # 모델 결과에서 필요한 정보 추출
    text = ' '.join(pill_result['text'])
    color = pill_result['color'][1]  # 구체적인 색상 이름 사용

    # 알약 식별 API 호출
    identification_params = {
        'serviceKey': API_KEY,
        'MARK_CODE_FRONT': text,
        'MARK_CODE_BACK': text,
        'COLOR_CLASS1': color,
        'COLOR_CLASS2': color,
        'ITEM_NAME': '',  # 제품명
        'ITEM_ENG_NAME': '',  # 제품 영문명
        'type': 'json'  # JSON 형식으로 응답 요청
    }

    identification_response = requests.get(PILL_IDENTIFICATION_API_URL, params=identification_params)
    identification_response.raise_for_status()
    identified_pills = identification_response.json().get('body', {}).get('items', [])

    # 가장 적합한 식별 결과 선택 (예: 첫 번째 결과)
    best_match = identified_pills[0] if identified_pills else None
    return best_match

def get_pill_info(identified_pill):
    if not identified_pill:
        return None

    # 알약 정보 API 호출
    info_params = {
        'serviceKey': API_KEY,
        'itemName': identified_pill.get('ITEM_NAME', ''),  # 식별된 알약의 이름 사용
        'type': 'json'  # JSON 형식으로 응답 요청
    }
    info_response = requests.get(PILL_INFO_API_URL, params=info_params)
    info_response.raise_for_status()
    pill_info = info_response.json().get('body', {}).get('items', [])

    return pill_info[0] if pill_info else None

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001, debug=True)